In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader, TensorDataset
from transformers import AutoModelForCausalLM, AutoTokenizer, AdamW
from transformers import AutoTokenizer, AutoConfig, AutoModelWithLMHead
import gdown
from transformers import GPT2LMHeadModel, GPT2Tokenizer, GPT2Config
import torch.nn as nn
from tqdm import tqdm
import numpy as np


## Extracting txt file

In [ ]:
!git clone https://github.com/AmirJodeiry2000/Deep_Learning_SUT_Fall_2023.git Ferdowsi_Poetry_Composer


fatal: destination path 'Ferdowsi_Poetry_Composer' already exists and is not an empty directory.


In [ ]:
import os
current_directory = '/content/Ferdowsi_Poetry_Composer'

file_path = os.path.join(current_directory, 'Ferdowsi_Poetry_Composer', 'ferdousi.txt')

# Open and read the content of the ferdousi.txt file
with open(file_path, 'r') as file:
    ferdowsi_poems = file.read()



## Dividing into verses.
## verse1 input , verse1 ouptut
## the same for both of them.

In [ ]:

input_verses = []
output_verses = []
poem_lines = ferdowsi_poems.strip().split('\n')

for i in range(2, len(poem_lines) - 2, 2):
    input_poetry = '    '.join([line    for line in poem_lines[i:i+2]])
    output_poetry = input_poetry
    input_verses.append(input_poetry)
    output_verses.append(output_poetry)

print(input_verses[-1])


از آن پس نمیرم که من زنده ام    که تخم سخن من پراگنده ام


### Adding model name and Tokenizer and it's considerations

In [ ]:
model_name_or_path = "HooshvareLab/gpt2-fa"

tokenizer = AutoTokenizer.from_pretrained(
    model_name_or_path,
    bos_token='<s>',
    eos_token='<eos>',
    pad_token='<pad>',
    unk_token='<unk>'
)
tokenizer.add_special_tokens({
    "bos_token": '<s>',
    "eos_token": '<eos>',
    "pad_token": '<pad>',
    "unk_token": '<unk>'
})
config = AutoConfig.from_pretrained(
    model_name_or_path,
    bos_token_id=tokenizer("<s>")["input_ids"][0],
    eos_token_id=tokenizer("<eos>")["input_ids"][0],
    pad_token_id=tokenizer("<pad>")["input_ids"][0],
    unk_token_id=tokenizer("<unk>")["input_ids"][0],
)

tokenizer.save_pretrained("/content/gpt2/")
config.save_pretrained("/content/gpt2/")

!wget "https://huggingface.co/HooshvareLab/gpt2-fa/resolve/main/pytorch_model.bin" -P /content/gpt2/
!wget "https://huggingface.co/HooshvareLab/gpt2-fa/resolve/main/tokenizer.json" -P /content/gpt2/

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


--2024-01-20 14:48:22--  https://huggingface.co/HooshvareLab/gpt2-fa/resolve/main/pytorch_model.bin
Resolving huggingface.co (huggingface.co)... 65.9.86.79, 65.9.86.62, 65.9.86.71, ...
Connecting to huggingface.co (huggingface.co)|65.9.86.79|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/HooshvareLab/gpt2-fa/46b0b806c740a0f0a9f056f5574c5fa896166fe844945fd3c849bf34365e5060?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27pytorch_model.bin%3B+filename%3D%22pytorch_model.bin%22%3B&response-content-type=application%2Foctet-stream&Expires=1706021302&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcwNjAyMTMwMn19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9Ib29zaHZhcmVMYWIvZ3B0Mi1mYS80NmIwYjgwNmM3NDBhMGYwYTlmMDU2ZjU1NzRjNWZhODk2MTY2ZmU4NDQ5NDVmZDNjODQ5YmYzNDM2NWU1MDYwP3Jlc3BvbnNlLWNvbnRlbnQtZGlzcG9zaXRpb249KiZyZXNwb25zZS1jb250ZW50LXR5cGU9KiJ9XX0_&Signature=Db0

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
    "/content/gpt2",
    bos_token='<s>',
    eos_token='<eos>',
    pad_token='<pad>'
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### Max seq of tokenized data

In [ ]:
max_seq = max([len(tokenizer.encode(text)) for text in input_verses])
max_seq

22

### Max seq of verses

In [ ]:
max_seq = max(len(poem) for poem in input_verses)
max_seq

67

### Writing A class for Dataset just to applying tokenization and returning input id and attention mask

In [ ]:
torch.manual_seed(42)


class MTGDataset(Dataset):

    def __init__(self, txt_list, tokenizer, max_length=1024):

        self.tokenizer = tokenizer  # the gpt2 tokenizer we instantiated
        self.input_ids = []
        self.attn_masks = []

        for txt in txt_list:
            """
            This loop will iterate through each entry in the flavour text corpus.
            For each bit of text it will prepend it with the start of text token,
            then append the end of text token and pad to the maximum length with the
            pad token.
            """

            encodings_dict = tokenizer('<s>' + txt + '<eos>',
                                       truncation=True,
                                       max_length=max_length,
                                       padding="max_length")

            """
            Each iteration then appends either the encoded tensor to a list,
            or the attention mask for that encoding to a list. The attention mask is
            a binary list of 1's or 0's which determine whether the langauge model
            should take that token into consideration or not.
            """
            self.input_ids.append(torch.tensor(encodings_dict['input_ids']))
            self.attn_masks.append(torch.tensor(encodings_dict['attention_mask']))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.attn_masks[idx]

### Creating tokenized input and ouptut ( the same )

In [ ]:
tokenized_input = MTGDataset(input_verses, tokenizer, max_length=max_seq)
tokenized_output = MTGDataset(output_verses, tokenizer, max_length=max_seq)


### Vocab size which will be used then

In [ ]:
vocab_size = len(tokenizer)
vocab_size

42002

In [ ]:
tokenized_input[10]

(tensor([    0,  2984,   297, 12519,  1204,   293,  1305,   293,  1079, 21147,
         17801, 37144,   330,  2496,  1010, 42001,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1]),
 tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]))

## Spliting into train and test

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import TensorDataset


input_ids = tokenized_input.input_ids
labels = tokenized_output.input_ids
attn_masks_input = tokenized_input.attn_masks
attn_masks_output = tokenized_output.attn_masks


train_input_ids, test_input_ids, train_labels, test_labels, train_attn_masks, test_attn_masks = train_test_split(
    input_ids, labels, attn_masks_input, test_size=0.2, random_state=42
)

train_input_ids = torch.stack(train_input_ids)
test_input_ids = torch.stack(test_input_ids)
train_labels = torch.stack(train_labels)
test_labels = torch.stack(test_labels)
train_attention_mask = torch.stack(train_attn_masks)
test_attention_mask = torch.stack(test_attn_masks)


train_dataset = TensorDataset(train_input_ids, train_attention_mask, train_labels)
test_dataset = TensorDataset(test_input_ids, test_attention_mask, test_labels)



## Building Dataloaders

### The max batch size is 32. more than that needs more poewerfull GPUs

In [ ]:
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

### Calling the model and config using the causalLm Architecture

In [ ]:
from transformers import AutoModelForCausalLM, AutoConfig

model_name_or_path = "HooshvareLab/gpt2-fa"

configuration = GPT2Config.from_pretrained(model_name_or_path, output_hidden_states=False)

model = AutoModelForCausalLM.from_pretrained(model_name_or_path, config=configuration)

model.resize_token_embeddings(len(tokenizer))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

print(device)


cuda


### Mounting the drive just for saving the model.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# model_save_path = "/content/drive/MyDrive/model_ferdowsi_same_verses.pt"
# model.load_state_dict(torch.load(model_save_path))


### Adding optimizer and criterion ( model has it's own criterion (CE))

In [ ]:
optimizer = torch.optim.AdamW(
    model.parameters(),
    lr = 1e-4
)
criterion = nn.CrossEntropyLoss()


## Train and test

In [ ]:
torch.cuda.empty_cache()
import gc
gc.collect()

23

In [ ]:
from tqdm import tqdm
import torch.nn.functional as F

def train_epoch(model, train_loader, optimizer, criterion, device):
    model.train()
    total_loss = 0.0

    for batch in tqdm(train_loader, desc="Training", leave=False):
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss=outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()

    avg_loss = total_loss / len(train_loader)
    return avg_loss

def evaluate(model, test_loader, criterion, device):
    model.eval()
    total_loss = 0.0
    total_tokens = 0

    with torch.no_grad():
        for batch in tqdm(test_loader, desc="Evaluating", leave=False):
            input_ids, attention_mask, labels = batch
            input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss=outputs.loss
            total_loss += loss.item()
            total_tokens += labels.numel()

    avg_loss = total_loss / len(test_loader)
    perplexity = torch.exp(torch.tensor(avg_loss))
    return avg_loss,perplexity.item()


num_epochs = 15


for epoch in range(num_epochs):
    print(f"\nEpoch {epoch + 1}/{num_epochs}")

    train_loss = train_epoch(model, train_loader, optimizer, criterion, device)
    print(f"Train Loss: {train_loss:}")

    test_loss,test_perplexity  = evaluate(model, test_loader, criterion, device)
    print(f'Test Perplexity: {test_perplexity}')
    print(f"Test Loss: {test_loss:}")

    model_save_path = "/content/drive/MyDrive/model_ferdowsi_same_verses.pt"
    torch.save(model.state_dict(), model_save_path)
    print(f"Model saved at: {model_save_path} IN EPOCH")



Epoch 1/15


Train Loss: 1.1417034618994768


Test Perplexity: 2.5954205989837646
Test Loss: 0.9537485772773767
Model saved at: /content/drive/MyDrive/model_ferdowsi_same_verses.pt IN EPOCH

Epoch 2/15


Train Loss: 0.8955685225443721


Test Perplexity: 2.380176544189453
Test Loss: 0.8671746809766223
Model saved at: /content/drive/MyDrive/model_ferdowsi_same_verses.pt IN EPOCH

Epoch 3/15


...

Test Perplexity: 2.7161295413970947
Test Loss: 0.9992079010347078
Model saved at: /content/drive/MyDrive/model_ferdowsi_same_verses.pt IN EPOCH

Epoch 14/15


Train Loss: 0.2661922644046312


Test Perplexity: 2.7857611179351807
Test Loss: 1.0245211143585646
Model saved at: /content/drive/MyDrive/model_ferdowsi_same_verses.pt IN EPOCH

Epoch 15/15


Train Loss: 0.2521922831017581


Test Perplexity: 2.8236982822418213
Test Loss: 1.0380474097092436
Model saved at: /content/drive/MyDrive/model_ferdowsi_same_verses.pt IN EPOCH


In [ ]:
import re
from IPython import display
import pandas as pd

def generator(poet, max_length=max_seq, num_return_sequences=3,temperature = 0.9):
    model.eval()
    print(prompt)

    input_ids = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)
    input_ids = input_ids.to(device)

    decoded_outputs = model.generate(
        input_ids,
        max_length=20,
        num_beams=10,
        no_repeat_ngram_size=2,
        top_k=150,
        pad_token_id=tokenizer.pad_token_id,
        attention_mask=torch.ones_like(input_ids),
        num_return_sequences =num_return_sequences
        )


    outputs = []

    for i, output in enumerate(decoded_outputs):
        o = tokenizer.decode(output, skip_special_tokens=False)
        o = o.replace("<s>", "").replace("<eos>", "\n")
        outputs.append(o)



    display.display(display.HTML("""
    <style>
    @import url("https://cdn.jsdelivr.net/gh/rastikerdar/vazir-font@v27.1.0/dist/font-face.css");

    table.xxx {
        margin-right: 15px;
        font-size: 14px;
        direction: rtl !important;
        width: 100%;
        display: flex;
    }
    table.xxx td {
        min-width: 300px !important;
        direction: rtl !important;
        text-align: right !important;
        font-family: "Vazir" !important;
    }
    </style>
    """.strip()))


    df = pd.DataFrame(outputs, columns=["generated"])
    df["generated"] = df["generated"].apply(lambda t: re.sub("\n+", "\n", t.replace("    ", "\n")))
    df["generated"] = df["generated"].apply(lambda t: "<p>" + t.replace("\n", "<br/>").replace("<|startoftext|>", "<br/>").strip() + "</p>")


    setup = {
        'border': 2,
        'show_dimensions': True,
        'escape': False,
        'justify': 'right',
        'classes': 'xxx'
    }
    display.display(display.HTML(df.to_html(**setup)))

### Some of the output samples

In [ ]:
prompt = "تو نیکی می کن و"
generator(prompt, num_return_sequences =2)
prompt = "توانا بود هر که"
generator(prompt, num_return_sequences =2)
prompt = "سالی که نکوست"
generator(prompt, num_return_sequences =2)
prompt = "چنین داد پاسخ  که نیک "
generator(prompt, num_return_sequences =2)

تو نیکی می کن و


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:396: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `150` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(


,generated
0,تو نیکی می کن و جام می برگزینکه من می خور و رامش گزین
1,تو نیکی می کن و جام و می خواستیبه می رامش و شادی بیاراستی


توانا بود هر که


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:396: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `150` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(


...

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:396: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `150` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(


,generated
0,سالی که نکوست بر ما زمان و زمینگذر کرد باید به دریای چین
1,سالی که نکوست بر ما زمان و زمینگذر کرد باید بداد آفرین


چنین داد پاسخ  که نیک 


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:396: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `150` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(


,generated
0,چنین داد پاسخ که نیک یران سپهرببینیم تا بر که گردد بمهر
1,چنین داد پاسخ که نیک یرانمهم از تخم شاهی و هم شیرم
